## Advent of Code 2022, Day 7

This code essentially reads a "conversation in terminal", and builds the tree of folders/files. My approach uses many recursive functions.

For detailed information, see: https://adventofcode.com/2022/day/7

Koen Groenland, Dec. 2022

In [23]:
MAXSIZE = 100000            # Part one: Maximum directory size to be eligible for deletion. 
NEED_TO_DELETE = 8381165    # Part two: Find smallest directory with at least this size. 


# The directory object can contain files and subdirectories. 
# New directories can be nested inside a directory within 'subdirs', forming a tree 
class directory :

    def __init__(self, name, parent ):
        self.parent = parent
        self.name = name
        self.subdirs = {}
        self.files = {}
        self.size = 0
        self.size_incl_subdirs = -1         # filled later
        self.total_maxsize_subdirs = -1     # filled later
        self.best_for_deletion = None         # filled later


    # Add a new directory by name (creates the directory on the spot)
    def add_dir( self, new_dir_name ):
        if new_dir_name not in self.subdirs :
            self.subdirs[new_dir_name] = directory(new_dir_name, self)



    # Add a file, which si a tuple (filesize, filename)
    def add_contents(self, filetuple ):
        if filetuple[1] not in self.files :
            self.files[filetuple[1]] = int(filetuple[0])
        self.size += int(filetuple[0])

    # Pretty print the whole tree, incl indent
    def print(self, indent = 0):
        print(" " * (3*indent) + '- ' + self.name, " (dir, ", self.size_incl_subdirs, ")" )
        print(self.files)

        for subdir in self.subdirs.items():
            subdir[1].print(indent=indent+1)

    # Recursively request subdirs to calculate their inclusive size. 
    # At the same time, collect the TOTAL SIZE of all subdirs that satisfy ( size_incl_subdirs < MAXSIZE )
    # Make sure to double-count if the parent also qualifies! 
    def set_sizes_incl_subdirs(self):

        self.total_maxsize_subdirs = 0
        self.size_incl_subdirs = self.size

        for subdir in self.subdirs.items():
            subdir[1].set_sizes_incl_subdirs()
            self.size_incl_subdirs += subdir[1].size_incl_subdirs

            # Transfer the total of MAXSIZE eligible:
            self.total_maxsize_subdirs += subdir[1].total_maxsize_subdirs


        # When this is done, check if the directory ITSELF still qualifies? 
        if(self.size_incl_subdirs < MAXSIZE ):
            self.total_maxsize_subdirs += self.size_incl_subdirs


    # Recusively request subdirs to set the "best for deletion" variable.
    # This is defined as the SMALLEST directory with at least size NEED_TO_DELETE
    def set_best_for_deletion(self):

        # Make sure all sizes are filled
        self.set_sizes_incl_subdirs()

        # Is this directory itself a good candiate?
        # If this directory is too small, we don't have to worry about subdirs. End the function. 

        if self.size_incl_subdirs >= NEED_TO_DELETE :
            self.best_for_deletion = self.size_incl_subdirs

        else : 
            return None

        # This directory is big enough? Then see if any of the subdirs are better?
        for subdir in self.subdirs.items():            
            subdir[1].set_best_for_deletion()
            b = subdir[1].best_for_deletion

            if (b is not None) and (b < self.best_for_deletion) :
                self.best_for_deletion = b

                
        



# Parse all lines of code, expanding the Directory object. 
def parse_lines( lines, rootdir ):
    
    currentdir = rootdir

    for l in lines:

#        print(" at line ", l , " we have: ")
#        rootdir.print()

        # check the type of line
        if l[0] == '$' :  # Deal with COMMAND

            if l[2] == 'c':
                # Change Directory command. Set current dir. Assume new directory has been seen before
                if l[5] == '/' : 
                    currentdir = rootdir
                elif l[5] == '.':
                    currentdir = currentdir.parent
                else :
                    newdirname = l.strip()[5:]
                    currentdir = currentdir.subdirs[newdirname]

            elif l[2] == 'l':
                # add info the the new dir! All folllowing lines are added to currentdir
                None

        elif l[0] == 'd': # reading new directory
            currentdir.add_dir(l.strip()[4:])

        elif l[0].isnumeric() : # adding a file
            filetuple = l.strip().split(" ")
#            print("filetuple: ", filetuple)
            currentdir.add_contents(filetuple)

    return rootdir
            

    
        


In [26]:
rootdir.best_for_deletion

In [29]:
''' MAIN FUNCTIONS '''

filename = 'input7.txt'
file = open(filename)
lines = file.readlines()
file.close()

# Start by making the root directory 
rootdir = directory("the root", "root")
parse_lines(lines, rootdir) # pass by reference
rootdir.set_sizes_incl_subdirs()

print("Rootdir total size: ", rootdir.size_incl_subdirs)
print("Rootdir sub-MAXSIZE total", rootdir.total_maxsize_subdirs)







Rootdir total size:  48690120
Rootdir sub-MAXSIZE total 1348005


In [30]:
'''
Part 2
'''

rootdir.set_best_for_deletion()
print("Best for deletion ", rootdir.best_for_deletion )

Best for deletion  12785886
